In [1]:
import sub.plot as pl
import sub.functions as sb
from global_variables import gparam
gl = gparam()

In [2]:
condition = {
    # TF current
    'cur_tf':{'tf': 50.0, 'turn': 16},
    
    # initial plasma profile
    'cur_ip':{'ip':-50.0, 'r0':0.65, 'z0':0.0, 'radius':0.3},
    
    # PF currents
    'cur_pf':{'hcult16':0.0,'pf17t12':0.5, 'pf26t18':1.0,'pf4_1ab3_cc2':1.0,'pf35_2':-0.5, },
    
    # number of coefficients
    'num_dpr':2, # dp/df
    'num_di2':2, # di2/df
    
    'resolution': gl.get_dmat_coarse(),
    }

In [3]:

# calculate plasma equilibrium
cond = sb.calc_equilibrium(condition, verbose=1)

0 loss:  0.2186516941089957
1 loss:  0.009014949944423978
2 loss:  0.002826016090498559
3 loss:  0.0014778165682179977
4 loss:  0.0005057392057805418
5 loss:  0.00035232492493329114
6 loss:  0.00027048940269654365
7 loss:  0.00020711349063106822
8 loss:  0.00014879817015760205
9 loss:  0.00019192161521404985


In [ ]:
# calculated parameters
cond.keys()

In [ ]:
# total flux profile
pl.d_contour(cond['flux'])

In [ ]:
# Plasma current profile
pl.d_contour(cond['jt'])

In [ ]:
# pressure
pl.d_contour(cond['pressure'])

In [ ]:
# poloidal current
pl.d_contour(cond['pol_current'])

In [ ]:
# vessel 
pl.d_heatmap(cond['vessel'])

In [ ]:
# domain
pl.d_heatmap(cond['domain'])